1) Importer les libraries required

In [ ]:
pip install keras

Python interpreter will be restarted.
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
Python interpreter will be restarted.


In [ ]:
pip install opencv-contrib-python

Python interpreter will be restarted.
  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (67.1 MB)
Python interpreter will be restarted.


In [ ]:
pip install tensorflow

Python interpreter will be restarted.
  Using cached tensorflow-2.10.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.1 MB)
  Using cached flatbuffers-22.10.26-py2.py3-none-any.whl (26 kB)
  Using cached termcolor-2.1.0-py3-none-any.whl (5.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.27.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.50.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached libclang-14.0.6-py2.py3-none-manylinux2010_x86_64.whl (14.1 MB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cache

In [ ]:
pip install bobo

Python interpreter will be restarted.
  Created wheel for bobo: filename=bobo-2.4.0-py3-none-any.whl size=17328 sha256=2d2f5dee78c3899a5db7e8e954d89a21dad05f903ddfdf67f7bb3eebfb46eb4d
  Stored in directory: /root/.cache/pip/wheels/37/3e/2a/113b14525a158f88f3df231d455336c8ef55fc83c7a2f16461
Successfully built bobo
Python interpreter will be restarted.


In [ ]:
#pip install opencv-contrib-python
#pip install keras
#pip install tensorflow
#pip install bobo

In [ ]:
import pyspark

from typing import Iterator, Any
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.functions import split
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, BinaryType
from pyspark.ml.image import ImageSchema 
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.ml.feature import PCA,StringIndexer  # Encoder une colonne
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import col, udf
# Librairie pour se connecter au service S3 d'AWS
#import bobo

In [ ]:
from keras import Model
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.layers import Dense

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import cv2
import os
import io
import sys
import time


2) Créer une SparkSession

In [ ]:
# create a SparkSession instance with the name moviedb with Hive support enabled
      # https://spark.apache.org/docs/latest/sql-data-sources-hive-tables.html
spark = SparkSession.builder.appName("imgdivers").enableHiveSupport().getOrCreate()

# create a SparkContext instance which allows the Spark Application to access 
# Spark Cluster with the help of a resource manager which is usually YARN or Mesos
sc = SparkContext.getOrCreate()

# create a SQLContext instance to access the SQL query engine built on top of Spark
sqlContext = SQLContext(spark)

/databricks/spark/python/pyspark/sql/context.py:117: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


3) Importer les fonctions

In [ ]:
def import_dossier(dir_path):
    """
    Envoyer un dataframe spark des chemins des images
    """
    Filelists = sc.wholeTextFiles(dir_path+"*/*.jpg").map(lambda x: x[0]).collect()
    Filelists_df = pd.DataFrame(Filelists, columns = ['image_path'])
    return spark.createDataFrame(data=Filelists_df)

In [ ]:
# Transformer the image in array and resize it.
def preprocessing_image_by_row(row,size):
    """
    Transformer une image dans un range d'un DataFrame vers un array et l'a redimentionne.
    """
    image_df = spark.read.format("image").load(row.__getitem__('image_path'))
    img = image_df.collect()[0][0]  # take first (first) row of the dataframe
    arr = ImageSchema.toNDArray(img)  #  (3024, 4032, 3)
    arr = cv2.resize(arr, size)  
    return arr
    

In [ ]:
def preprocessing_image(img_path,size): 
    """
    Transformer une image vers un array et l'a redimentionne via son chemin.
    taille d'input pour le modèle VGG16 : arr = arr.reshape(1,224,224,3) 
    """
    image_df = spark.read.format("image").load(img_path)
    img = image_df.collect()[0][0]  # take first (first) row of the dataframe
    arr = ImageSchema.toNDArray(img)  #  (3024, 4032, 3)
    arr = cv2.resize(arr, size)    #vgg16 (224,224)
    return arr                       #.flatten().tolist()   # envoyer une list :1200

In [ ]:
def endoder_type_fruit(df,inputcol, outputcol):
    qualification_indexer = StringIndexer(inputCol=inputcol, outputCol=outputcol)
    #Fits a model to the input dataset with optional parameters.
    return qualification_indexer.fit(df).transform(df)
    

In [ ]:
def model_prediction(df, model,size):
    row_list = df.collect()
    model_feature_list = []
    
    for i in range(df.count()):
        arr = preprocessing_image_by_row(row_list[i],size).reshape(1,224,224,3)  # préparer les inputs pour le modèle
        model_feature = model.predict(arr)
        model_feature_np = np.array(model_feature)
        model_feature_np = cv2.resize(model_feature_np, (20,20)) # new line  
        model_feature_list.append(model_feature_np.flatten())  
    return np.array(model_feature_list)

In [ ]:
def list_preprocessed_image(df,size):
    row_list = df.collect()
    list_arr =[] 
    for i in range(df.count()):
        arr = preprocessing_image_by_row(row_list[i],size)
        arr = np.asarray(arr).flatten().tolist()   #array-->list
        list_arr.append(arr)
    return list_arr

4) Exécution programme

In [ ]:
#variable to hold the main directory path
dirPath="s3://bucketoc/Training/"
df = import_dossier(dirPath)

#df_rdd1 = import_dossier(dirPath)
#df_rdd2 = df_rdd1.rdd.map(lambda x: (x[0], extrait_type_fruit(x[0])))
#df=df_rdd2.toDF(["image_path","Type_fruit"]   )
df.printSchema()
df.show(5, truncate=False)

root
 |-- image_path: string (nullable = true)

+------------------------------------------+
|image_path                                |
+------------------------------------------+
|s3://bucketoc/Training/Banana/327_100.jpg |
|s3://bucketoc/Training/Banana/90_100.jpg  |
|s3://bucketoc/Training/Banana/r_80_100.jpg|
|s3://bucketoc/Training/Corn/58_100.jpg    |
|s3://bucketoc/Training/Corn/r2_86_100.jpg |
+------------------------------------------+
only showing top 5 rows



In [ ]:
df=df.withColumn('Type_fruit', split(df['image_path'], '/').getItem(4))
df.show(truncate=False)

+------------------------------------------+----------+
|image_path                                |Type_fruit|
+------------------------------------------+----------+
|s3://bucketoc/Training/Banana/327_100.jpg |Banana    |
|s3://bucketoc/Training/Banana/90_100.jpg  |Banana    |
|s3://bucketoc/Training/Banana/r_80_100.jpg|Banana    |
|s3://bucketoc/Training/Corn/58_100.jpg    |Corn      |
|s3://bucketoc/Training/Corn/r2_86_100.jpg |Corn      |
|s3://bucketoc/Training/Corn/r_118_100.jpg |Corn      |
|s3://bucketoc/Training/Corn/r_139_100.jpg |Corn      |
|s3://bucketoc/Training/Orange/121_100.jpg |Orange    |
|s3://bucketoc/Training/Orange/213_100.jpg |Orange    |
|s3://bucketoc/Training/Orange/228_100.jpg |Orange    |
|s3://bucketoc/Training/Orange/229_100.jpg |Orange    |
+------------------------------------------+----------+



In [ ]:
df = endoder_type_fruit(df,inputcol="Type_fruit", outputcol="Label")
df.printSchema()
df.show(5, truncate=False)

root
 |-- image_path: string (nullable = true)
 |-- Type_fruit: string (nullable = true)
 |-- Label: double (nullable = false)

+------------------------------------------+----------+-----+
|image_path                                |Type_fruit|Label|
+------------------------------------------+----------+-----+
|s3://bucketoc/Training/Banana/327_100.jpg |Banana    |2.0  |
|s3://bucketoc/Training/Banana/90_100.jpg  |Banana    |2.0  |
|s3://bucketoc/Training/Banana/r_80_100.jpg|Banana    |2.0  |
|s3://bucketoc/Training/Corn/58_100.jpg    |Corn      |0.0  |
|s3://bucketoc/Training/Corn/r2_86_100.jpg |Corn      |0.0  |
+------------------------------------------+----------+-----+
only showing top 5 rows



In [ ]:
# read le contenent of a column
df_col1= df.rdd.map(lambda r: r.image_path).collect()   # ['...jpg', '....jpg']
preprocessed_image_df = pd.DataFrame({ 'image_paths': df_col1, 'image_preprocessed':list_preprocessed_image(df,(20,20))})
sparkDF=spark.createDataFrame(preprocessed_image_df) 
df = df.join(sparkDF, sparkDF.image_paths==df.image_path,"right")
df= df.drop(df.image_paths)
df.show()

+--------------------+----------+-----+--------------------+
|          image_path|Type_fruit|Label|  image_preprocessed|
+--------------------+----------+-----+--------------------+
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|
|s3://bucketoc/Tra...|    Orange|  1.0|[254, 253, 255, 2...|
|s3://bucketoc/Tra...|    Orange|  1.0|[255, 255, 252, 2...|
|s3://bucketoc/Tra...|    Orange|  1.0|[255, 254, 253, 2...|
|s3://bucketoc/Tra...|    Orange|  1.0|[255, 254, 253, 2...|
+--------------------+----------+-----+--------------------+



In [ ]:
df.printSchema()

root
 |-- image_path: string (nullable = true)
 |-- Type_fruit: string (nullable = true)
 |-- Label: double (nullable = true)
 |-- image_preprocessed: array (nullable = true)
 |    |-- element: long (containsNull = true)



5) Modèle

In [ ]:
from keras import Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
# load model
model_vgg16 = VGG16()
# remove the output layer
model_vgg16 = Model(inputs = model_vgg16.inputs,
                    outputs = model_vgg16.layers[-2].output)
# summarize the model
model_vgg16.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
vgg16_feature_list_np = model_prediction(df, model_vgg16,(224,224))

1/1 [==============================] - 1s 551ms/step


In [ ]:
vgg16_feature_list_np.shape

Out[67]: (11, 400)

In [ ]:
# read le contenent of a column
df_vgg16 = df.rdd.map(lambda r: r.image_path).collect()   # ['...jpg', '....jpg']
df_2 = pd.DataFrame({ 'image_paths': df_vgg16, 'image_vgg16':vgg16_feature_list_np.tolist()})
sparkDF_2=spark.createDataFrame(df_2) 
df = df.join(sparkDF_2, sparkDF_2.image_paths==df.image_path,"right")
df= df.drop(df.image_paths)
df.show()


+--------------------+----------+-----+--------------------+--------------------+
|          image_path|Type_fruit|Label|  image_preprocessed|         image_vgg16|
+--------------------+----------+-----+--------------------+--------------------+
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[0.54681098461151...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[0.0, 0.0, 0.0, 1...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[2.52145075798034...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.11277621239423...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.0, 0.313618630...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.0, 0.229949980...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.0, 0.099657982...|
|s3://bucketoc/Tra...|    Orange|  1.0|[254, 253, 255, 2...|[0.16559562087059...|
|s3://bucketoc/Tra...|    Orange|  1.0|[255, 255, 252, 2...|[0.27285405993461...|
|s3://bucketoc/T

5 ) ACP

In [ ]:
# Transform 'features' into dense vectors and add its in features_max (new column 'vectors')
transform_into_dense_vector = udf(lambda v: Vectors.dense(v), VectorUDT())
df = df.withColumn('vectors', transform_into_dense_vector("image_vgg16"))

In [ ]:
standardizer = StandardScaler(inputCol="vectors", outputCol="scaledFeatures",
                                  withStd=True, withMean=True)
model_std = standardizer.fit(df)
df = model_std.transform(df)
df= df.drop(df.vectors)


In [ ]:
pca = PCA(k=50, inputCol='scaledFeatures',  # input column name
          outputCol='features_pca')  # output column name

# Fit on features_max
model_pca = pca.fit(df)
df = model_pca.transform(df)
df = df.filter(df.features_pca.isNotNull())

In [ ]:
df.show()

+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+
|          image_path|Type_fruit|Label|  image_preprocessed|         image_vgg16|      scaledFeatures|        features_pca|
+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[0.54681098461151...|[0.22479788478684...|[-9.1505103638971...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[0.0, 0.0, 0.0, 1...|[-0.5238663058903...|[-0.0619107306066...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[2.52145075798034...|[2.92836845928527...|[-20.535707841105...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.11277621239423...|[-0.3694591813887...|[7.49631873546192...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.0, 0.313618630...|[-0.5238663058903...|[5.50170291673157...|
|s3://bu

6) Ecrire Spark DataFrame en Parquet file et le sauvegarder dans Amazon S3.

In [ ]:
df.write.parquet(path='s3://bucketoc/Training/parquet/imgdivers.parquet"', mode='overwrite')

In [ ]:
parqDF = spark.read.parquet('s3://bucketoc/Training/parquet/imgdivers.parquet"/part-00000-tid-6440133969391812144-807f1a06-bec8-4c58-9d73-fdf89aaf846f-1272-1-c000.snappy.parquet')
parqDF.show()

+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+
|          image_path|Type_fruit|Label|  image_preprocessed|         image_vgg16|      scaledFeatures|        features_pca|
+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.0, 0.313618630...|[-0.5238663058903...|[5.50170291673158...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[2.52145075798034...|[2.92836845928527...|[-20.535707841105...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.11277621239423...|[-0.3694591813887...|[7.49631873546191...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[0.0, 0.0, 0.0, 1...|[-0.5238663058903...|[-0.0619107306066...|
|s3://bucketoc/Tra...|    Banana|  2.0|[255, 255, 255, 2...|[0.54681098461151...|[0.22479788478684...|[-9.1505103638971...|
+-------

In [ ]:
parqDF2 = spark.read.parquet('s3://bucketoc/Training/parquet/imgdivers.parquet"/part-00001-tid-6440133969391812144-807f1a06-bec8-4c58-9d73-fdf89aaf846f-1273-1-c000.snappy.parquet')
parqDF2.show()

+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+
|          image_path|Type_fruit|Label|  image_preprocessed|         image_vgg16|      scaledFeatures|        features_pca|
+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+
|s3://bucketoc/Tra...|    Orange|  1.0|[254, 253, 255, 2...|[0.16559562087059...|[-0.2971416881198...|[-0.0074391715089...|
|s3://bucketoc/Tra...|    Orange|  1.0|[255, 255, 252, 2...|[0.27285405993461...|[-0.1502892020963...|[1.07016331836980...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.0, 0.229949980...|[-0.5238663058903...|[16.0982737878794...|
|s3://bucketoc/Tra...|      Corn|  0.0|[255, 255, 255, 2...|[0.0, 0.099657982...|[-0.5238663058903...|[2.61580573903764...|
|s3://bucketoc/Tra...|    Orange|  1.0|[255, 254, 253, 2...|[0.29348516464233...|[-0.1220422031452...|[-0.7559757635897...|
|s3://bu

In [ ]:
parqDF2.select(['image_path', 'Type_fruit']).show(5, truncate=False)

+-----------------------------------------+----------+
|image_path                               |Type_fruit|
+-----------------------------------------+----------+
|s3://bucketoc/Training/Orange/121_100.jpg|Orange    |
|s3://bucketoc/Training/Orange/213_100.jpg|Orange    |
|s3://bucketoc/Training/Corn/r_118_100.jpg|Corn      |
|s3://bucketoc/Training/Corn/r_139_100.jpg|Corn      |
|s3://bucketoc/Training/Orange/228_100.jpg|Orange    |
+-----------------------------------------+----------+
only showing top 5 rows



7) Test a single image

In [ ]:
#### Test avec une seule image ####

# Visualize one of the images
image_test = spark.read.format("image").load("s3://bucketoc/test-multiple_fruits/tomatoes3.jpg" )
arr = preprocessing_image("s3://bucketoc/test-multiple_fruits/tomatoes3.jpg",(224,224))
arr = arr.reshape(1,224,224,3) 

In [ ]:
vgg16_feature = model_vgg16.predict(arr)
vgg16_feature.shape

1/1 [==============================] - 5s 5s/step
Out[59]: (1, 4096)